In [1]:
import json
import altair as alt
from altair import expr, datum
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import requests

In [21]:
LOCAL = True

if LOCAL:
    local_suffix = "_local"
else:
    local_suffix = ""

In [22]:
%%capture pwd
!pwd

In [98]:
# uid = "2021-05-05-which-firms-and-industries-have-been-most-affected-by-covid-update"  # article unique ID
uid = pwd.stdout.split("/")[-1].split("\r")[0]
eco_git_home = (
    "https://raw.githubusercontent.com/EconomicsObservatory/ECOvisualisations/main/"
)
eco_git_path = eco_git_home + "articles/" + uid + "/data/"
vega_embed = requests.get(eco_git_home + "guidelines/html/vega-embed.html").text
colors = json.loads(
    requests.get(eco_git_home + "guidelines/colors/eco-colors.json").content
)
category_color = json.loads(
    requests.get(eco_git_home + "guidelines/colors/eco-category-color.json").content
)
hue_color = json.loads(
    requests.get(eco_git_home + "guidelines/colors/eco-single-hue-color.json").content
)
mhue_color = json.loads(
    requests.get(eco_git_home + "guidelines/colors/eco-multi-hue-color.json").content
)
div_color = json.loads(
    requests.get(eco_git_home + "guidelines/colors/eco-diverging-color.json").content
)
config = json.loads(
    requests.get(eco_git_home + "guidelines/charts/eco-global-config.json").content
)
yConfig = alt.Axis(
    title="%",
    domain=False,
    ticks=False,
    labelAlign="left",
    labelBaseline="middle",
    labelPadding=-5,
    labelOffset=-10,
    titleX=35,
    titleY=-6,
    titleAngle=0,
    titleFontSize=11,
    titleAlign="left",
    tickCount=5,
    format=".0f",
)
xConfig = alt.Axis(
    grid=False,
    title="",
    titleAnchor="end",
    titleY=-15,
    labelColor=colors["eco-gray"],
    titleColor=colors["eco-gray"],
    tickColor=colors["eco-gray"],
    domainColor=colors["eco-gray"],
)
height = config["height"]
width = config["width"]
height, width

(300, 500)

Common

In [154]:
recessions = [
    {"start": "1990-01-01", "end": "1993-01-01", "event": "recession"},
    {"start": "2008-01-01", "end": "2011-01-01", "event": "recession"},
    {"start": "2020-01-01", "end": "2021-04-01", "event": "covid"},
]
color_dict = {"covid": "#F3F4D8", "recession": "#E2E2E2"}
dr = pd.DataFrame(recessions)
dr["color"] = [color_dict[i] for i in dr["event"]]

# Fig 1

In [155]:
df = (
    pd.read_excel("raw/Fig1.xls")
    .dropna(how="all")
    .drop("recessions", axis=1)
    .set_index("qdate")
    .stack()
    .reset_index()
)
df.columns = ["date", "students", "value"]
df["students"] = df["students"].str.replace("student_", "")
df["students"] = df["students"].str[:2] + "-" + df["students"].str[2:]

In [156]:
f = "fig1_employees"
f1 = eco_git_path + f + ".csv"
df.to_csv("data/" + f + ".csv")
f += local_suffix
open("visualisation/" + f + ".html", "w").write(
    vega_embed.replace(
        "JSON_PATH", f1.replace("/data/", "/visualisation/").replace(".csv", ".json")
    )
)
if LOCAL:
    f1 = df
df.head()

,date,students,value
0,1985-01-01,16-17,52.815990
1,1985-01-01,18-20,14.839300
2,1985-01-01,21-24,4.291038
3,1986-01-01,16-17,52.805893
4,1986-01-01,18-20,15.169523


In [157]:
base = alt.Chart(f1).encode(
    x=alt.X(
        "date:T",
        title="",
        axis=alt.Axis(
            grid=False,
            title="",
            titleAnchor="end",
            titleY=-15,
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
        ),
    ),
    y=alt.Y(
        "value:Q",
        title="",
        axis=alt.Axis(
            title="%",
            domain=False,
            ticks=False,
            labelAlign="left",
            labelBaseline="middle",
            labelPadding=-5,
            labelOffset=-10,
            titleX=20,
            titleY=28,
            titleAngle=0,
            titleFontSize=10,
            titleAlign="left",
            tickCount=5,
            format=".0f",
        ),
    ),
    color=alt.Color(
        "students:N",
        scale=alt.Scale(
            range=[
                colors["eco-turquiose"],
                colors["eco-light-blue"],
                colors["eco-blue"],
            ]
        ),
        legend=None,
    ),
)
line = base.mark_line()
text = (
    base.mark_text(align="left", xOffset=5)
    .encode(text="students:N")
    .transform_filter("year(datum.date)==2021")
)
yAxis = (
    alt.Chart(pd.DataFrame([{"value": 0}]))
    .mark_rule(size=0)
    .encode(
        x=alt.X("value:Q"),
    )
)
rect = (
    alt.Chart(dr)
    .mark_rect(blend="darken")
    .encode(
        x="start:T",
        x2="end:T",
        color=alt.Color("color:N", scale=None),
    )
)
layer = (
    (line + text + rect)
    .configure_view(stroke=None)
    .properties(title="")
    .properties(height=300, width=400)
)
layer.save("visualisation/" + f + ".json")
layer

alt.LayerChart(...)

# Fig 2

In [183]:
df = (
    pd.read_excel("raw/Fig2.xls")
    .dropna(how="all")
    .drop("recessions", axis=1)
    .set_index("qdate")
    .stack()
    .reset_index()
)
df.columns = ["date", "students", "value"]
df["students"] = df["students"].str.replace("l1_stud_", "")
df["students"] = df["students"].str[:2] + "-" + df["students"].str[2:]

In [185]:
f = "fig2_part_time"
f2 = eco_git_path + f + ".csv"
df.to_csv("data/" + f + ".csv")
f += local_suffix
open("visualisation/" + f + ".html", "w").write(
    vega_embed.replace(
        "JSON_PATH", f2.replace("/data/", "/visualisation/").replace(".csv", ".json")
    )
)
if LOCAL:
    f2 = df
df.head()

,date,students,value
0,1985-01-01,16-17,27.994228
1,1985-01-01,18-20,23.756548
2,1985-01-01,21-24,13.920683
3,1986-01-01,16-17,30.125587
4,1986-01-01,18-20,23.870945


In [187]:
base = alt.Chart(f2).encode(
    x=alt.X(
        "date:T",
        title="",
        axis=alt.Axis(
            grid=False,
            title="",
            titleAnchor="end",
            titleY=-15,
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
        ),
    ),
    y=alt.Y(
        "value:Q",
        title="",
        axis=alt.Axis(
            title="%",
            domain=False,
            ticks=False,
            labelAlign="left",
            labelBaseline="middle",
            labelPadding=-5,
            labelOffset=-10,
            titleX=20,
            titleY=28,
            titleAngle=0,
            titleFontSize=10,
            titleAlign="left",
            tickCount=5,
            format=".0f",
        ),
    ),
    color=alt.Color(
        "students:N",
        scale=alt.Scale(
            range=[
                colors["eco-turquiose"],
                colors["eco-light-blue"],
                colors["eco-blue"],
            ]
        ),
        legend=None,
    ),
)
line = base.mark_line()
text = (
    base.mark_text(align="left", xOffset=5)
    .encode(text="students:N")
    .transform_filter("year(datum.date)==2021")
)
yAxis = (
    alt.Chart(pd.DataFrame([{"value": 0}]))
    .mark_rule(size=0)
    .encode(
        x=alt.X("value:Q"),
    )
)
rect = (
    alt.Chart(dr)
    .mark_rect(blend="darken")
    .encode(
        x="start:T",
        x2="end:T",
        color=alt.Color("color:N", scale=None),
    )
)
layer = (
    (line + text + rect)
    .configure_view(stroke=None)
    .properties(title="")
    .properties(height=300, width=400)
)
layer.save("visualisation/" + f + ".json")
layer

alt.LayerChart(...)

# Fig 3

In [190]:
df = (
    pd.read_excel("raw/Fig3.xls")
    .dropna(how="all")
    .drop("recessions", axis=1)
    .set_index("qdate")
    .stack()
    .reset_index()
)
df.columns = ["date", "students", "value"]
df["students"] = df["students"].str.replace("l3_stud_", "")
df["students"] = df["students"].str[:2] + "-" + df["students"].str[2:]

In [192]:
f = "fig3_inactivity"
f3 = eco_git_path + f + ".csv"
df.to_csv("data/" + f + ".csv")
f += local_suffix
open("visualisation/" + f + ".html", "w").write(
    vega_embed.replace(
        "JSON_PATH", f3.replace("/data/", "/visualisation/").replace(".csv", ".json")
    )
)
if LOCAL:
    f3 = df
df.head()

,date,students,value
0,1985-01-01,16-17,66.374710
1,1985-01-01,18-20,73.326401
2,1985-01-01,21-24,84.456047
3,1986-01-01,16-17,63.082581
4,1986-01-01,18-20,73.329002


In [202]:
base = alt.Chart(f3).encode(
    x=alt.X(
        "date:T",
        title="",
        axis=alt.Axis(
            grid=False,
            title="",
            titleAnchor="end",
            titleY=-15,
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
        ),
    ),
    y=alt.Y(
        "value:Q",
        title="",
        axis=alt.Axis(
            title="%",
            domain=False,
            ticks=False,
            labelAlign="left",
            labelBaseline="middle",
            labelPadding=-5,
            labelOffset=-10,
            titleX=25,
            titleY=-5,
            titleAngle=0,
            titleFontSize=10,
            titleAlign="left",
            tickCount=5,
            format=".0f",
        ),
        scale=alt.Scale(domain=[40, 100]),
    ),
    color=alt.Color(
        "students:N",
        scale=alt.Scale(
            range=[
                colors["eco-turquiose"],
                colors["eco-light-blue"],
                colors["eco-blue"],
            ]
        ),
        legend=None,
    ),
)
line = base.mark_line()
text = (
    base.mark_text(align="left", xOffset=5)
    .encode(text="students:N")
    .transform_filter("year(datum.date)==2021")
)
yAxis = (
    alt.Chart(pd.DataFrame([{"value": 0}]))
    .mark_rule(size=0)
    .encode(
        x=alt.X("value:Q"),
    )
)
rect = (
    alt.Chart(dr)
    .mark_rect(blend="darken")
    .encode(
        x="start:T",
        x2="end:T",
        color=alt.Color("color:N", scale=None),
    )
)
layer = (
    (line + text + rect)
    .configure_view(stroke=None)
    .properties(title="")
    .properties(height=300, width=400)
)
layer.save("visualisation/" + f + ".json")
layer

alt.LayerChart(...)